# Описание проекта

Вас пригласили на стажировку в отдел маркетинговой аналитики Яндекс.Афиши. Первое задание: помочь маркетологам снизить расходы — отказаться от невыгодных источников трафика и перераспределить бюджет.
Есть данные Яндекс.Афиши с июня 2017 по конец мая 2018 года:
- лог сервера с данными о посещениях сайта Яндекс.Афиши,  
- выгрузка всех заказов за этот период,  
- статистика рекламных расходов.  

Цели к изучению:
- как клиенты пользуются сервисом,  
- когда делают первые покупки на сайте,   
- сколько денег приносит компании каждый клиент,  
- когда расходы на привлечение клиента окупаются.    

Выручка измеряется в условных единицах — у.е.

<h1> Описание данных</h1>    

`visits_log.csv` - хранит лог сервера с информацией о посещениях сайта.    
Uid — уникальный идентификатор пользователя,     
Device — категория устройства пользователя,    
Start Ts — дата и время начала сессии,   
End Ts — дата и время окончания сессии,    
Source Id — идентификатор источника перехода на сайт.   

`orders_log.csv` - содержит информацию о заказах.    
Uid — уникальный идентификатор пользователя,      
Buy Ts — дата и время заказа,    
Revenue — сумма заказа.   

`costs.csv` - хранит информацию о расходах на маркетинг.    
source_id — идентификатор рекламного источника,    
dt — дата проведения рекламной кампании,     
costs — расходы на эту кампанию.

# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

# Изучение и предобработка данных

## Импорт данных

In [2]:
visits = pd.read_csv('visits_log.csv')
orders = pd.read_csv('orders_log.csv')
costs = pd.read_csv('costs.csv')

In [3]:
visits['uid'].duplicated()

KeyError: 'uid'

## Предобработка log

In [ ]:
visits.head(3)

In [ ]:
visits.info()
visits.shape

Пропуски отсутсвуют. Размер таблицы 359400 на 5.

In [ ]:
# найдем дубликаты
dup = visits.duplicated().sum()
if dup == 0:
    print('Дубликаты отсутствуют')
else:
    print(f'Есть дубликаты в кол-ве: {dup}')

In [ ]:
# переведём End Ts и Start Ts в формат datetime
visits['End Ts'] = pd.to_datetime(visits['End Ts'], format = '%Y-%m-%d %H:%M:%S')
visits['Start Ts'] = pd.to_datetime(visits['Start Ts'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
# выведем названия столбцов
visits.columns

In [ ]:
# переименуем названия столбцов
visits.columns = 'device', 'end_ts', 'source_id', 'start_ts', 'uid'

In [ ]:
print(visits['start_ts'].describe())
print(visits['end_ts'].describe())

**Вывод:**   
есть сессии, который закончились 2018-06-01. Не будем удалять эти данные иначе можно потерять посещения предыдущего дня.

In [ ]:
visits['device'] = visits['device'].astype('category')

In [ ]:
print('Пользовтаели посещают платформу Я.Афиши с устройств:',visits['device'].unique())

In [ ]:
visits.groupby(['device']).agg({'uid':'count'})

**Вывод:**   
с десктопа посещают в 2.7 раза больше, чем с touch устройств

In [ ]:
visits.info()

## Предобработка orders

In [ ]:
orders.head(3)

In [ ]:
orders.info()
orders.shape

Пропуски отсутсвуют. Размер таблицы 50415 на 3.

In [ ]:
# найдем дубликаты
dup = orders.duplicated().sum()
if dup == 0:
    print('Дубликаты отсутствуют')
else:
    print(f'есть дубликаты в кол-ве: {dup}')

In [ ]:
# переведём Buy Ts в формат datetime
orders['Buy Ts'] = pd.to_datetime(orders['Buy Ts'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
orders.columns

In [ ]:
# переименуем названия столбцов
orders.columns = 'buy_ts', 'revenue', 'uid'

In [ ]:
print(orders['buy_ts'].describe())

In [ ]:
orders.info()

## Предобработка costs

In [ ]:
costs.head(3)

In [ ]:
costs.info()
costs.shape

Пропуски отсутсвуют. Размер таблицы 2542 на 3.

In [ ]:
# найдем дубликаты
dup = costs.duplicated().sum()
if dup == 0:
    print('Дубликаты отсутствуют')
else:
    print(f'есть дубликаты в кол-ве: {dup}')

In [ ]:
# переведём Buy Ts в формат datetime
costs['dt'] = pd.to_datetime(costs['dt'], format = '%Y-%m-%d')

In [ ]:
costs.columns

In [ ]:
print(costs['dt'].describe())

In [ ]:
costs.info()

In [ ]:
costs['source_id'].unique()

# Рассчитаем метрики и построим графики

Метрики для расчёта:
 - продуктовые
 - метрики эл. коммерции.
 - маркетинговые
 ## Продуктовые метрики

In [ ]:
visits['date'] = visits['start_ts'].dt.date
visits['day_number'] = visits['start_ts'].dt.day
visits['month_number'] = visits['start_ts'].dt.month
visits['year'] = visits['start_ts'].dt.year
visits['week_number'] = visits['start_ts'].astype('datetime64[W]')
visits['day_of_week'] = visits['start_ts'].dt.day_name()
visits['month_name'] = visits['start_ts'].astype('datetime64[M]')

visits.head(5)

### DAU, WAU, MAU

In [ ]:
dau_total = visits.groupby(['date']).agg({'uid':'nunique'}).mean()
wau_total = visits.groupby(['week_number']).agg({'uid':'nunique'}).mean()
mau_total = visits.groupby(['month_number']).agg({'uid':'nunique'}).mean()

print(f'Ежeдневная аудитория DAU: {dau_total[0]:.0f} человек')
print(f'Еженедельная аудитория WAU: {wau_total[0]:.0f} человек')
print(f'Ежемесячная аудитория WAU: {mau_total[0]:.0f} человек')

In [ ]:
sticky_wau = (dau_total/wau_total)
sticky_mau = (dau_total/mau_total)
print(f' Sticky Factor (недельный) {sticky_wau[0]:.2%}')
print(f' Sticky Factor (месячный) {sticky_mau[0]:.2%}')

**Вывод:**  
15.59% посетителей обращаются к Я.Афише в течении недели, а 3.91% посетителей обращается к Я.Афише в течении месяца. 

Данному сервису сложно удержать внимание посетителей на таком длинном промежутке времени.  Предполагаю, что данный результат достаточно низкий для подобного сервиса.

In [ ]:
# стиль для графиков
plt.rcParams['figure.figsize']=(15,5)
plt.style.use('bmh')
dau_dynamic = visits.groupby(['date']).agg({'uid':'nunique'}).reset_index()
sns.lineplot(x = 'date' , y = 'uid', data = dau_dynamic)
plt.title('Динамика DAU')
plt.xlabel('Дни')
plt.ylabel('Кол-во уникальных посетителей')
plt.show()
register_matplotlib_converters()

In [ ]:
print(f'Ежeдневная аудитория DAU: {dau_total[0]:.0f} человек')

In [ ]:
wau_dynamic = visits.groupby(['week_number']).agg({'uid':'nunique'}).reset_index()
sns.lineplot(x = 'week_number' , y = 'uid', data = wau_dynamic)
plt.rcParams['figure.figsize']=(15,5)
plt.title('Динамика WAU')
plt.xlabel('Номер недели')
plt.ylabel('Кол-во уникальных посетителей')
plt.show()

In [ ]:
print(f'Ежeдневная аудитория DAU: {wau_total[0]:.0f} человек')

**Вывод:**   
наблюдаются всплески посещений на: 12, 22 и 47 неделе. Возможно, что это связано с маркетинговыми активностями.

In [ ]:
mau_dynamic = visits.groupby(['month_name']).agg({'uid':'nunique'}).reset_index()
sns.lineplot(x = 'month_name' , y = 'uid', data = mau_dynamic)
plt.rcParams['figure.figsize']=(15,5)
plt.title('Динамика MAU')
plt.xlabel('Месяц')
plt.ylabel('Кол-во уникальных посетителей')
plt.show()

In [ ]:
print(f'Ежемесячная аудитория WAU: {mau_total[0]:.0f} человек')

In [ ]:
cost_mrkt = costs.groupby(['dt']).agg({'costs':'sum'}).reset_index()
cost_mrkt['dt']=cost_mrkt['dt'].dt.strftime('%Y-%m')
sns.lineplot(x = 'dt' , y = 'costs', data = cost_mrkt)
plt.rcParams['figure.figsize']=(15,5)
plt.title('Динамика маркетинговых расходов')
plt.xlabel('Месяц')
plt.ylabel('Расходы, у.е')
plt.show()

**Вывод:**   
c 09-2017, наблюдаются всплески по маркетинговым активносятм, посещаемость на эту дату также растёт. Соответсвенно посещаемость сильно зависит от маркетинговых активностей. Это видно даже без наложения графиков - <u>WAU</u> и <u>Динамака</u> маркетинговых расходов.        
DAU: 908 человек;    
WAU: 5825 человек;    
WAU: 23228 человек.        


### Сколько сессий в день

In [ ]:
session_count = visits.groupby('date').agg({'uid': 'nunique'}).mean()
print(f'Среднее количество сессий в день за период с июня 2017 по конец мая 2018 г.: {session_count[0]:.2f}')

In [ ]:
session_month = visits.groupby('date').agg({'uid': 'count'}).reset_index()
sns.lineplot(x='date', y='uid', data = session_month)
plt.title('Динамика посещаемости')
plt.xlabel('Номер месяца')
plt.ylabel('Кол-во уникальных посетителей')
plt.show()

### Сколько длится одна сессия?¶

In [ ]:
visits['session_duration_minutes'] = (visits['end_ts'] - visits['start_ts']).dt.total_seconds()/60
visits['session_duration_minutes'].describe()

In [ ]:
# убираем отрицательные сессии
visits = visits.query('session_duration_minutes >=0')

In [ ]:
print(visits['session_duration_minutes'].describe())
print('mode',visits['session_duration_minutes'].mode())

**Вывод:**   
    продолжительность одной сессии:   
        - в среднем 10 мин.    
        - медиана 5 мин.    
        - мода 1 мин.    
Так как среднее больше медианы, имеем гистограмму смещенную вправо, что вполне естественно для большинства сайтов. А также означает, что больше всего данных лежит примерно до 10 минут.

In [ ]:
visits['session_duration_minutes'].hist(bins=20, range=(-5, 60))
plt.title('Среднее одной сессии')
plt.xlabel('Минуты')
plt.ylabel('Кол-во посетителей')
plt.show()

**Вывод:**  
ожидания подтвердились гистограммой, следовательно основная часть пользователей проводит до 10 минут на сайте Я.Афиши. Возможно этого достаточно для совершения целевого действия на платформе.

### Retention Rate

In [ ]:
first_visit = visits.groupby(['uid'])['start_ts'].min().reset_index()
first_visit.columns = 'uid', 'first_visit'
first_visit.head(3)

In [ ]:
visits = visits.merge(first_visit)
visits.head(2)

In [ ]:
visits['first_visit_month'] = visits['first_visit'].astype('datetime64[M]')
visits['visit_month'] = visits['start_ts'].astype('datetime64[M]')
visits.head(2)

In [ ]:
visits['lifetime'] = (visits['visit_month'] - visits['first_visit_month']) / np.timedelta64(1, 'M')
visits['lifetime'] = visits['lifetime'].round().astype('int')

In [ ]:
cohort_visits = visits.groupby(['first_visit_month', 'lifetime']).agg({'uid': 'nunique'}).reset_index()
f_uid = cohort_visits[cohort_visits['lifetime'] == 0][['first_visit_month', 'uid']]
f_uid.columns = ['first_visit_month', 'f_uid']
f_uid.head(3)

In [ ]:
cohort_visits = cohort_visits.merge(f_uid)
cohort_visits["retention"] = cohort_visits["uid"] / cohort_visits["f_uid"]
visits_retention_pivot = cohort_visits.pivot_table(
    index="first_visit_month", columns="lifetime", values="retention", aggfunc="sum"
)
visits_retention_pivot.mean() * 100

visits_retention_pivot.index = visits_retention_pivot.index.strftime("%m/%Y")
visits_retention_pivot

In [ ]:
sns.set(style='white')
plt.figure(figsize=(13, 9))
plt.title('Когорты: Коэффициент удержания', size=14)
ax =sns.heatmap(visits_retention_pivot, annot=True, fmt='.1%', linewidths=0.2, linecolor='gray', cmap = 'viridis',vmax=0.15)
ax.set_ylabel('Месяц когорты')
ax.set_xlabel('Месяц жизни когорты')
plt.show()

**Вывод:**  
90-95% пользователей отваливается, видимо связано с их особенностью. Мало кто возвращается ежемесячно для покупки билетов или просмотра афиш. Всего скорее сервис используется для посещения знаковых событий, таких как кинопремьеры и выставки.

## Метрики e-commerce
### Время совершения первой покупки

In [ ]:
first_visit = visits.groupby('uid').agg({'start_ts':'min'})\
                .rename(columns={'start_ts':'first_visit_date'})\
                .reset_index()

first_order = orders.groupby('uid').agg({'buy_ts':'min'})\
                .rename(columns={'buy_ts':'first_purchase_date'})\
                .reset_index()

buying_time = first_visit.merge(first_order, on='uid', how='inner')
buying_time.head(3)

In [ ]:
buying_time['buying_time'] = (buying_time['first_purchase_date'] - buying_time['first_visit_date']).dt.seconds / 60
buying_time.head()

In [ ]:
buying_time['buying_time'].describe()

In [ ]:
buying_time['buying_time'].mode()

**Вывод:**   
в большинстве случаев заказ делают в день первого визита.

In [ ]:
buying_time['buying_time'].hist()
plt.rcParams['figure.figsize']=(8,5)
plt.title('Среднее время первой покупки')
plt.xlabel('Минуты')
plt.ylabel('Кол-во покупок')
plt.show()

**Вывод:**   
большое количество покупок совершается в первый день. Далее количество покупок резко снижается и начинает немного отрастать на 1000 минуте. Предполагаю, что это пользователи, которые ожидали какого-то события и решили вернуться для покупки на Я.Афише.

### Cреднее количество покупок на одного покупателя

#### Когорта среднего количесва покупок на одного покупателя

In [ ]:
# найдем число покупателей в каждой когорте
orders['order_month'] = orders['buy_ts'].astype('datetime64[M]')
orders_per_month = orders.groupby('order_month').agg({'revenue': 'count'}).rename(columns={'revenue':'purchases'})
orders_per_month.reset_index()

In [ ]:
# добавим в информацию о покупках месяц первой покупки
first_orders = orders.groupby('uid').agg({'order_month': 'min'}).reset_index()
first_orders.columns = ['uid', 'first_order_month']
first_orders.head()

In [ ]:
# посчитаем количесво новых покупателей за каждый месяц
cohort_sizes = first_orders.groupby('first_order_month').agg({'uid': 'nunique'}).reset_index()
cohort_sizes.columns = ['first_order_month', 'n_buyers']
cohort_sizes.head()

In [ ]:
orders_new = pd.merge(orders,first_orders, on='uid')
orders_new.head()

In [ ]:
# Сгруппируем таблицу заказов по месяцу первой покупки и месяцу каждого заказа и сложим выручку. 
# Сбросим индекс методом reset_index()
cohorts = orders_new.groupby(['first_order_month','order_month']).agg({'revenue': 'count'}).reset_index().rename(columns={'revenue':'order_count'})

cohorts.head()

In [ ]:
report = pd.merge(cohort_sizes, cohorts, on='first_order_month')
report.head()

In [ ]:
report['time'] = (report['order_month'] - report['first_order_month']) / np.timedelta64(1, 'M')
report['time'] = report['time'].round().astype('int')
report.head()

In [ ]:
report['mean_order'] = report['order_count'] / report['n_buyers']
report

In [ ]:
output_order_mean = report.pivot_table(
        index='first_order_month', 
        columns='time', 
        values='mean_order', 
        aggfunc='mean').round(2)
output_order_mean.index = output_order_mean.index.strftime("%m/%Y")
output_order_mean=output_order_mean.cumsum(axis=1).round(2)
# output_order_mean

In [ ]:
plt.figure(figsize=(13, 9))
plt.title('Среднее количество покупок на одного покупателя', size=14)
ax=sns.heatmap(output_order_mean, annot=True, fmt='.2f', linewidths=0.2, linecolor='gray', vmin=1, vmax=2.5, cmap="viridis")
ax.set_ylabel('Месяц когорты')
ax.set_xlabel('Месяц жизни когорты')
plt.show()

#### Среднее количество покупкок в месяц

In [ ]:
orders_mean = pd.merge(orders_per_month, cohort_sizes, left_on='order_month', right_on = 'first_order_month')

In [ ]:
orders_mean['mean_orders'] = orders_mean['purchases'] / orders_mean['n_buyers']
f"В среднем в 1 месяц покупают - {round(orders_mean['mean_orders'].mean(),2)} раза"

**Вывод:** 
Среднее количество покупок во всех когортах имеет рост. 
Наиболее успешные когорты по среднему количеству покупок: 06/2017, 07/2017, 08/2017, 11/2017.    
06/2017 - среднее кол-во покупок на 8 месяце жизни достигает 2. Возможно в этой когорте самые лояльные клиенты.    
---
В среднем в месяц покупают 1,34 раза.

### Средний чек

In [ ]:
print('Средний чек равен:',orders['revenue'].mean().round(),'y.e')

In [ ]:
orders['revenue'].mode()

In [ ]:
orders

In [ ]:
mean_purchase_by_month = orders.groupby('order_month').agg({'revenue': 'mean'}).reset_index()
mean_purchase_by_month['order_month'] = mean_purchase_by_month['order_month']
mean_purchase_by_month

In [ ]:
ax = sns.lineplot(x='order_month', y='revenue', data = mean_purchase_by_month)
plt.title('Средний чек в динамике')
plt.xlabel('Месяц')
plt.ylabel('Средний чек, y.e')
plt.grid()
plt.show()

**Вывод:**  
наиболее часто встречается стоимость 1.8 y.e.   
Средний чек = 5 y.e Средний чек резко вырастет в 12 месяце, а в 1 месяце снижается до показателей 7 месяца. Падение составляет около 43%.

### LTV
1. Найдём даты первой покупки;
2. Найдём число новых покупателей на каждую дату;
3. Добавим дату первой покупки в таблицу с заказами;
4. Сгруппируем таблицу и рассчитаем выручку

In [ ]:
# месяц первой покупки для каждого клиента
orders['order_month'] = orders['buy_ts'].astype('datetime64[M]')
orders['buy_ts'] = pd.to_datetime(orders['order_month'], format = '%Y-%m-%d')
first_orders = orders.groupby('uid').agg({'buy_ts': 'min'}).reset_index()
first_orders.columns = ['uid', 'first_order_month']
first_orders.head()

In [ ]:
# Рассчитаем кол-во новых покупателей за каждый месяц
cohort_sizes = first_orders.groupby('first_order_month').agg({'uid': 'nunique'}).reset_index()
cohort_sizes.columns = ['first_order_month', 'n_buyers']
cohort_sizes.head(13)

In [ ]:
cohort_sizes.head()

In [ ]:
# добавим месяц первой покукп в таблицу с заказами. Объединяем по uid клиента.
orders_new = pd.merge(orders, first_orders, on = 'uid')
orders_new.head()

In [ ]:
# сгруппируем таблицу заказов по месяцу первой покупки и месяцу заказа, затем сложим выручку.
cohorts = orders_new.groupby(['first_order_month', 'order_month']).agg({'revenue': 'sum'}).reset_index()
cohorts.head()

In [ ]:
# добавим количество первых покупок в каждый месяц
final = pd.merge(cohorts,cohort_sizes, on = 'first_order_month')
final.head()

LTV рассчитывается из валовой прибыли, но так как у нас нет доступа к этим данным, будем считать что 100% выручка остаётся в компании.

In [ ]:
final['time'] = (final['order_month'] - final['first_order_month']) / np.timedelta64(1, 'M')
final['time'] = final['time'].round().astype('int')

In [ ]:
final.head()

In [ ]:
final['ltv'] = final['revenue']/final['n_buyers']
ltv_output = final.pivot_table(
        index='first_order_month', 
        columns='time', 
        values='ltv', 
        aggfunc='mean').round(2)
ltv_output.index = ltv_output.index.strftime("%m/%Y")
ltv_output= ltv_output.cumsum(axis=1).round(2)

In [ ]:
# визуализируем LTV
sns.set(style='white')
plt.figure(figsize=(13, 9))
plt.title('Когорты: LTV', size=14)
ax=sns.heatmap(ltv_output, annot=True, fmt='.2f', linewidths=0.2, linecolor='black', vmin=1,  cmap="viridis", vmax=12)
ax.set_ylabel('Месяц когорты')
ax.set_xlabel('Месяц жизни когорты')
plt.show()

In [ ]:
print('Cредний LTV по когортам за 6 месяцев:', ltv_output[6].mean())

**Вывод:**  
 06/2017 и 09/2017 - являются самыми прибыльными когортами. В январе и феврале, на работу можно не выходить =)
 Очень сильный рост наблюдается в когортах 12/2017, 09/2017, 12/2017 на третьем месяце жизни. 
 Пожизненная стоимость клиента растёт по каждой кагорте, а значит есть потенциал для роста в каждой из них.

## Маркетинговые метрики

#### Сумма расходов

In [ ]:
print('Сумма маркетинговых расходов:', costs['costs'].sum(), 'y.e')

#### Расходы по источникам

In [ ]:
source_costs = costs.groupby(['source_id']).agg({'costs':'sum'}).reset_index()
plt.figure(figsize=(8, 4))
sns.barplot(x='source_id', y='costs', data = source_costs)
plt.title('Расходы по источника, y.e')
plt.xlabel('Источники')
plt.ylabel('Расходы, y.e')
plt.show()

**Вывод:**  
максимальная сумма расходов приходится на источник №3.

#### Расходы по месяцам

In [ ]:
# производили данные расчёты в разделе DAU, WAU, MAU
plt.rcParams['figure.figsize']=(15,5)
cost_mrkt = costs.groupby(['dt']).agg({'costs':'sum'}).reset_index()
cost_mrkt['dt']=cost_mrkt['dt'].dt.strftime('%Y-%m')
sns.lineplot(x = 'dt' , y = 'costs', data = cost_mrkt)
plt.title('Динамика маркетинговых расходов')
plt.xlabel('Месяц')
plt.ylabel('Расходы, у.е')
plt.show()

In [ ]:
monthly_costs=cost_mrkt.groupby('dt').agg({'costs':'sum'}).reset_index()
monthly_costs['dt'] = monthly_costs['dt'].astype('datetime64[M]')
monthly_costs_mean = monthly_costs.mean().round()
print('Среднемесячные траты:', monthly_costs_mean)

**Вывод:**  
больше всего маркетинговых расходов приходится на 10, 11 и 12 месяцы. В среднем расходуется около 27 428 y.e

### CAC

#### CAC по всему проекту

In [ ]:
# день первого визита и его рекламный источник
first_visit_day = visits.groupby(['uid','source_id']).agg({'start_ts':'min'}).reset_index()
first_visit_day.columns = 'uid','source_id','start_ts'
first_visit_day.head()

In [ ]:
# # Маркетинговые расходы по месяцам
monthly_costs.columns = 'month', 'costs'
monthly_costs

In [ ]:
report_cac = pd.merge(report, 
                      monthly_costs, 
                      left_on = 'first_order_month',
                      right_on = 'month')
report_cac['cac'] = report_cac['costs'] / report_cac['n_buyers']
report_cac.head()

In [ ]:
report_cac['cac'] = report_cac['costs'] / report_cac['n_buyers']
report_cac['month'] = report_cac['month'].dt.strftime('%Y-%m')
report_cac.head()

In [ ]:
# график динамики CAC
sns.lineplot(x = 'month' , y = 'cac', data = report_cac)
plt.rcParams['figure.figsize']=(15,5)
plt.title('Динамика CAC')
plt.xlabel('Месяц')
plt.ylabel('CAC, у.е')
plt.grid()
plt.show()

**Вывод:**  
самоё дорогое привлечение приходится на 2017-08 и составляет 9.7 y.e, затем в 2017-09 и 2017-08 CAC резко снижается до 8.4 y.e. Далее растёт до 10 y.e, пик данного роста приходится на 2017-12, затем снижается до рекодно низких значений в 7.5 y.e, на дату 2018-05.    
Возможная причина роста на 2017-08, это запуск новых рекламных кампаний на новом(ых) источнике(ах).  

#### Средний CAC по сему проекту

In [ ]:
# Средний CAC по сему проекту
cac_mean  = report_cac['cac'].mean()
print(f'Средний CAC по всему проекту =  {cac_mean:.2f} y.e')

#### CAC по источникам

In [ ]:
visits_by_source = visits.sort_values('start_ts').groupby('uid').first()
visits_by_source = visits_by_source['source_id']
orders_by_source = orders.merge(visits_by_source, on = 'uid', how = 'left')
orders_by_source.head()

In [ ]:
orders_by_source['order_month'] = orders_by_source['order_month'].astype('datetime64[D]')
cac_per_source_id = (orders_by_source.groupby(['source_id'], as_index = False)
                     .agg({'uid':'nunique'})
                     .rename(columns = {'uid':'n_buyers'})
                     .merge(costs.groupby('source_id', as_index = False)
                            .agg({'costs': 'sum'}), on = 'source_id')
                     .assign(cac = lambda row: row['costs'] / row['n_buyers'])
                    )
cac_per_source_id

**Вывод:**   
самое дорогое привлечение приходится на источник №3. Стоимость привлечения одного клиента = 13.49 y.e Данный канал также привлёк 10473 клиентов. Так как это самый дорогой канал, можно отказаться от данного канала и распределить его бюджет на более привлекательные.

  
 
Над каналом 9 и 10 явно необходимо ещё поэксперементировать, так как стоимость клиента низкая, а бюджет израсходаван в разы ниже, по сравнению с другими источниками.

### ROMI

In [ ]:
romi=pd.merge(final, monthly_costs, left_on='first_order_month', right_on='month')
romi['cac'] = romi['costs'] / romi['n_buyers']
romi['romi'] = romi['ltv'] / romi['cac']
output = romi.pivot_table(
        index='first_order_month', 
        columns='time', 
        values='romi', 
        aggfunc='mean')

output=output.cumsum(axis=1).round(2)
output.index = output.index.strftime("%m/%Y")

In [ ]:
sns.set(style='white')
plt.figure(figsize=(13, 9))
plt.title('Когорты: ROMI', size=14, weight = 'bold')
ax=sns.heatmap(output, annot=True, fmt='.2f', linewidths=1, linecolor='black', cmap = 'hot')
ax.set_ylabel('Месяц когорты')
ax.set_xlabel('Месяц жизни когорты');

**Вывод:**  
окупились когорты:    
- 06/2017 на 6 месяце
- 09/2017 на 3 месяце.

### Рассчитаем ROMI по источникам рекламы

In [ ]:
# найдем рекламный источник первого визита для каждого клиента
# и объеденим с таблицей orders

first_uid_source = visits.groupby(['uid'])['source_id'].min()
first_uid_source.name = 'first_uid_source'
first_uid_source = orders.join(first_uid_source,on='uid')
first_uid_source.head()

In [ ]:
# выведем выручку по каждому источнику
revenue_source=first_uid_source.groupby(['first_uid_source'])['revenue'].sum().reset_index()
revenue_source

In [ ]:
romi_source = cac_per_source_id.merge(revenue_source, left_on='source_id', right_on='first_uid_source')
romi_source['romi']=romi_source['revenue']/romi_source['costs']
romi_source.drop('first_uid_source', axis=1, inplace=True)

romi_source

In [ ]:
output = romi_source.sort_values(by='romi', ascending=False).reset_index()

ax=output.plot(y='romi', x='source_id', kind='bar', figsize=(8, 5), title='ROMI по источникам', legend=False, rot=360)
plt.axhline(y=1, color='r', linestyle='--')
ax.text(.99, 0.22, 'порог окупаемости',
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='red', fontsize=12)
plt.xlabel('Источники рекламы')
plt.ylabel('ROMI')
plt.show()

**Вывод:**  
источник №1 окупается почти в 4 раза. Источник 2 выходит в ноль, при том, он самый дорогой. 
Остальные источники с первого взгляда не являются прибыльными, но это совсем не так. Если сложить ROMI данных источников, то данные каналы имеют право на жизнь.

# Общий вывод

## Выполненные работы     

    1. Данные изучены и подготовлены к анализу     
    2. Рассчитаны метрики     
    3. Построены необходимые графики  

**Рассчитал и отобразил:**

Продукт:
- Посещаемость DAU, WAU, MAU
- Количество сессий и их продолжительность
- Retantion rate

E-commerce:
- Время совершения первой покупки
- Среднее количество покупок за 6 месяцев
- Средний чек
- LTV

Маркетинг:
- Сумма расходов
- Расходы по источникам
- Расходы по месяцам
- CAC
- ROMI

## Итоговый вывод

Яндекс.Афиша с июня 2017 по конец мая 2018 года    
Всего посещений за: 359400     
Всего покупок: 50415     
Конверсия в покупку = 14%     
Business season начинается в марте и завершается в сентябре.     
Бизнесу свойственны событийные транзакции.
Если включить в рассчёты аренду офиса, пресонал и др. постоянные расходы, то бизнес сильно работает в минус.
- - -
Ежeдневная аудитория DAU: 908 человек;     
Еженедельная аудитория WAU: 5825 человек;     
Ежемесячная аудитория WAU: 23228 человек;    
Среднее количество сессий в день: 987 сессий;    
В среднем сессия длится: 11 минут;    
Когорта 06/2017 имеет самый высокий Retention Rate на протяжении всего периода;     
Заказ совершается в первые 150 минут;     
В среднем покупают в день/неделя/месяц, раз: 138.502 / 969.519 / 4201.250;     
Средний чек составляет: 5 y.e;     
Больше всего маркетинговых расходов приходится на 10, 11 и 12 месяцы. В среднем расходуется около 27 428 y.e     
окупились когорты:    
- 06/2017 на 6 месяце
- 09/2017 на 3 месяце.    

Средний CAC 9.32 y.e больше среднего чека 5 y.e, что сигнализирует о проблемах.     


ROMI: источник №1 окупается почти в 4 раза. Источник 2 выходит в ноль, при том он самый дорогой. 
Остальные источники с первого взгляда не являются прибыльными, но это совсем не так. Если сложить ROMI данных истоников, то данные каналы имеют право на жизнь.

**Предложение по маркетинговой стратегии**    
Из всех маркетинговых стратегий, самая удачная была в сентябре 2017 г. её можно взять за ориентир или вернуться к её более подробному изучению. 


Если канал №1 является email рассылка по своей базе, то все же не стоит рассматривать данный канал как привлечение новых пользователей, он будет лучше работать на удержание. Бизнес нужно масштабировать привлекая новых клиентов. А для этого нужно тестировать новые каналы привлечения, отказываясь от неэффективных.    
Поэтому шаги должны быть такие:
1. Определить max CAC, который нельзя превышать. Глобально CAC не должен превышать средний чек = 5 y.e.
2. Отказаться от канала №3 и перекинуть его бюджет на дургие, более привлекательные каналы.
3. Заняться поиском, ранее неиспользуемых каналов привлечения.